In [4]:
# Attempt to generalize Conway's nimbers to characteristic p>2
'''
At each stage, we identify p^(p^k) with a root of the polynomial x^p - x - p^(p^(k-1))
This makes $\\mathbb{N}$ into a field isomorphic to the union of GF(p^(p^k)) over all k
'''

def deg(N : int, p : int = 2) -> int:
    '''
    Returns the largest D such that 
    p ** (p ** D) <= N
    under the p_nimber multiplication, the ordinal p^(p^D)
    is a field extension of Z/pZ of degree p^D
    '''
    D = 0
    if N < p:
        return 0
    while N // (p ** (p ** D)) != 0 :
        D += 1
    return D-1

def p_decomp(N : int, p : int = 2,  as_array = False, degree : int = -1) -> list:
    '''
    returns dict {'prime':p,'degree':degree,0:a_0,1:a_1,...,p-1:a_{p-1}} in the decomposition
    N = sum_{k=0}^{p-1} a_k * p^(k*(p^L)) , 
    where D=deg(N,p) and each a_k < p^(p^L).
    This is well-defined and unique by euclidean algorithm.
    '''
    if degree == -1:
        degree = deg(N,p)
    if p == 2:
        power = 1 << (1 << degree)
        decomp = {'prime':p,'degree':degree,0:N % power,1:N // power}
        if as_array:
            return [decomp[k] for k in range(p)] 
        return decomp
    else:
        power = p ** (p ** degree)
        decomp = {'prime':p,'degree':degree,0:N % power}
        for k in range(1,p):
            N = N // power
            decomp[k] = (N % power)
        if as_array:
            return [decomp[k] for k in range(p)]
        return decomp
    
def p_combine(decomp : dict) -> int:
    '''
    the inverse of p_decomp
    '''
    p = decomp['prime']
    degree = decomp['degree']
    if p == 2:
        return decomp[0] + (decomp[1] << (1 << degree))
    else:
        power = p ** (p ** degree)
        N = decomp[0]
        for k in range(1,p):
            N += decomp[k] * power
            power *= p ** (p ** degree)
        return N

In [ ]:
# sanity check
dim = 3
N = 100
k = 1256
for n in range(N):
    decomp = p_decomp(k*n,dim,as_array=False)
    decomp_array = p_decomp(k*n,dim,as_array=True)
    print(f'{k*n} = {decomp}\n = {decomp_array}')
    print(f'{k*n} = {p_combine(decomp)}')

In [90]:
def int_log(N : int, base : int) -> int:
    '''
    returns floor(log_{base}(N)) 
    '''
    if N < base:
        return 0
    # for the highest power 'exp' s.t. base^exp <= N, find
    # the largest power of 2 that is less than or equal to exp
    total = 1 << deg(N, base)
    # now divide N to recursively find the binary expansion of exp
    N = N // (base ** total)
    while N != 0:
        total += 1 << deg(N, base)
        N = N // (base ** (1 << deg(N, base)))
    return total - 1

def nim_sum(N : int, M: int, p : int = 2) -> int:
    '''
    Returns the p-nimber sum of N and M
    '''
    # trivial cases
    if N == 0:
        return M
    elif M == 0:
        return N
    # if p = 2, then the sum is just the bitwise xor
    if p == 2:
        return N ^ M
    # otherwise, we do sum modulo p in each "p-bit" 
    if N < p and M < p:
        return (N + M) % p
    else:
        return ((N + M) % p) + p * nim_sum(N//p, M//p,p)

def nim_neg(N : int, p : int = 2) -> int:
    '''
    Returns the p-nimber negation of N
    '''
    if N == 0:
        return 0
    if p == 2:
        return N
    else:
        return (-N) % p + p * nim_neg(N // p, p)

In [ ]:
p = 3
N = 27
for n in range(N):
    print(f'nim_neg({n}) = {nim_neg(n,p)}')
for n in range(N):
    print(f'{n} + {nim_neg(n,p)} = {nim_sum(n,nim_neg(n,p),p)}')
    


In [78]:
# encode p_nimber multiplication recursively via matrices
# special case of pi(D):=p**(p**D - 1) is easier to multiply
# because its decomp is [0,0,0,...,0,pi(D-1)]
import numpy as np

def pi_mult(N: int, D : int, p : int = 2) -> int:
    '''returns p**(p**D - 1) * N   w.r.t p-nimber multiplication
        whenever degree(N) < D
        otherwise the answer is incorrect, but we can recursively
        make sure the degree is less than D
    '''
    if D == 0:
        return N
    decomp_N = p_decomp(N,p,as_array=True,degree = D-1)
    decomp_product = {'prime':p,'degree':D-1}
    for k in range(p):
        decomp_product[k] = 0
    for k in range(p):
        for j in range(p):
            term1 = int((k == p-1) and (j == 0)) * pi_mult(decomp_N[j],D-1,p)
            term2 = int((k > 0) and (j == k)) * pi_mult(decomp_N[j],D-1,p)
            term3 = int(j == k+1) * pi_mult(pi_mult(decomp_N[j],D-1,p),D-1,p)
            sum = nim_sum(term3, nim_sum(term1,term2,p), p)
            decomp_product[k] =  nim_sum(decomp_product[k], sum, p)
    return p_combine(decomp_product)

def nim_mult_matrix(N : int, p : int =2, D=-1) -> np.ndarray:
    '''returns the multiplication by N matrix'''
    if D == -1:
        D = deg(N,p)
    M = np.zeros((p,p),dtype=object)
    decomp_N = p_decomp(N,p,as_array=True,degree = D)
    for k in range(p):
        for j in range(p):
            condition1 = (k >= j)
            condition2 = (0 < k) and (k <= j)
            condition3 =  (j > k) and (0 <= p - (j - k)) and (p - (j - k) < p)

            if condition1:
                M[k,j] = nim_sum(M[k,j],decomp_N[k-j],p) 

            if condition2:
                M[k,j] = nim_sum(M[k,j],decomp_N[(p-1) - (j-k)],p)

            if condition3:
                M[k,j] = nim_sum(M[k,j], pi_mult(decomp_N[p - (j - k)],D,p), p)
    return M

def nim_product(N : int, M : int, p : int = 2) -> int:
    '''returns N * M w.r.t p-nimber multiplication'''
    # base case D = 0
    if N < p and M < p:
        return N * M % p
    # now inductively multiply assuming multiplication is already
    # defined for degrees less than D = max(deg(N,p),deg(M,p))
    D = max(deg(N,p),deg(M,p))
    bigger = max(N,M); smaller = min(N,M)

    # write min as a vector
    decomp_smaller = p_decomp(smaller,p,as_array=True,degree = D)
    # then write bigger as a multiplication matrix
    bigger_matrix = nim_mult_matrix(bigger,p,D)

    # initialize the decomposition dictionary for the product
    decomp_product = {'prime':p,'degree':D}
    for k in range(p):
        decomp_product[k] = 0

    # do the matrix multiplication of mult_matrix(max) and decomp_min
    for i in range(p):
        for j in range(p):
            decomp_product[i] = nim_sum(decomp_product[i], nim_product(bigger_matrix[i,j], decomp_smaller[j], p), p)
    
    # finally, recombine the decomposition vector into a single integer
    return p_combine(decomp_product)
    

    

In [174]:
# find determinant of nimber matrix
def nim_det(M : np.ndarray, p : int = 2) -> int:
    '''returns the nim_determinant of square matrix M'''
    dim = M.shape[0]
    if dim == 1:
        return M[0,0]
    det = 0
    for j in range(dim):
        # find the minor matrix
        minor = np.zeros((dim-1,dim-1),dtype=object)
        for i in range(1,dim):
            for k in range(dim):
                if k < j:
                    minor[i-1,k] = M[i,k]
                elif k > j:
                    minor[i-1,k-1] = M[i,k]
        # recursively find the determinant
        if j % 2 == 0:
            det = nim_sum(det, nim_product(M[0,j], nim_det(minor,p), p), p)
        else:
            det = nim_sum(det, nim_neg(nim_product(M[0,j], nim_det(minor,p), p), p), p)
    return det

# find inverse of N in GF(p)
def p_inv(N : int, prime : int = 2) -> int:
    '''
    returns the inverse of N in GF(prime)
    may give incorrect results if prime is not prime
    '''
    p =  prime
    if N == 0:
        raise ValueError('0 is not invertible')
    if N == 1:
        return 1
    if N < 0 or N >= p:
        raise ValueError(f'{N} is not in GF({p})')
    else:
        # perform the euclidean algorithm
        if N == 1:
            return 1
        a = 1; b = 0;c = 0; d = 1
        # p = a*p + b*N
        # N = c*p + d*N 
        while N > 1:
            q = p // N
            r = p % N
            m = a - c * q
            n = b - d * q
            a = c; b = d; c = m; d = n
            p = N; N = r
        return d % prime
    

def nim_matrix_mult(M : np.ndarray, N : np.ndarray, p : int = 2) -> np.ndarray:
    '''returns the matrix product of M and N'''
    # make sure the dimensions are compatible
    if M.shape[1] != N.shape[0]:
        raise ValueError('Matrix dimensions are not compatible')

    # create the product matrix
    product = np.zeros((M.shape[0],N.shape[1]),dtype=object)
    for i in range(M.shape[0]):
        for j in range(N.shape[1]):
            for k in range(M.shape[1]):
                product[i,j] = nim_sum(product[i,j], nim_product(M[i,k],N[k,j],p), p)
    return product

# find inverse of nimber matrix
def nim_matrix_inv(M : np.ndarray, p : int = 2) -> np.ndarray:
    # base case
    if M.shape[0] == 1:
        return np.array([p_inv(M[0,0],p)],dtype=object)
    # create the augmented matrix with identity
    augmented = np.zeros((M.shape[0],2*M.shape[1]),dtype=object)
    for i in range(M.shape[0]):
        for j in range(M.shape[1]):
            augmented[i,j] = M[i,j]
    for i in range(M.shape[0]):
        augmented[i,M.shape[1]+i] = 1
    # now do row reduction
    for i in range(M.shape[0]):
        # find the pivot
        pivot = i
        while pivot < M.shape[0] and augmented[pivot,i] == 0:
            pivot += 1
        # if no pivot, then the matrix is not invertible
        if pivot == M.shape[0]:
            return None
        # swap rows
        augmented[[i,pivot]] = augmented[[pivot,i]]
        # divide by pivot using recursive call to inverse
        # base case
        if augmented[i,i] < p:
            pivot_inv = p_inv(augmented[i,i],p)
        else:
            pivot_degree = deg(augmented[i,i],p)
            pivot_mult_matrix = nim_mult_matrix(augmented[i,i],p)
            pivot_inv_decomp = {'prime':p,'degree':pivot_degree}
            for k in range(p):
                pivot_inv_decomp[k] = nim_matrix_inv(pivot_mult_matrix,p)[k,0]
            pivot_inv = p_combine(pivot_inv_decomp)
        for j in range(2*M.shape[1]):
            augmented[i,j] = nim_product(augmented[i,j],pivot_inv,p)
        # now do row operations
        for j in range(M.shape[0]):
            if j != i:
                factor = augmented[j,i]
                for k in range(2*M.shape[1]):
                    augmented[j,k] = nim_sum(augmented[j,k], nim_neg(nim_product(factor,augmented[i,k],p),p),p)
    # return the inverse
    return augmented[:,M.shape[1]:]


In [ ]:
p = 3
N = 10
for n in range(1,N):
    #print(f'nim_mult_matrix({n},{p})=\n{nim_mult_matrix(n,p)}')
    #print(f'nim_matrix_inv({n},{p})=\n{nim_matrix_inv(nim_mult_matrix(n,p),p)}\n')
    print(f'{nim_mult_matrix(n,p)}\n*\n{nim_matrix_inv(nim_mult_matrix(n,p),p)}\n={nim_matrix_mult(nim_mult_matrix(n,p),nim_matrix_inv(nim_mult_matrix(n,p),p),p)}\n')